In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input/train-dataset-kriti'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
import numpy as np
import pandas as pd

In [ ]:
train_data_path = '/kaggle/input/train-dataset-kriti/train.csv'


In [ ]:
train_data_csv = pd.read_csv(train_data_path)
train_data_csv.head()

In [ ]:
unique_values = train_data_csv['Categories'].unique()

In [ ]:
print(unique_values)

In [ ]:
tags_lists = [eval(tag_str) for tag_str in unique_values]

# Step 2: Flatten the list of lists
flattened_tags = [tag for sublist in tags_lists for tag in sublist]

# Step 3: Get the unique tags
unique_tags = np.unique(flattened_tags)

print("Unique tags (combined):")
print(unique_tags)

In [ ]:
print(len(unique_tags))

In [ ]:
import nltk
from string import punctuation
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words('english') + list(punctuation)
print(stopwords.words('english'))

In [ ]:
# from nltk.tokenize import word_tokenize
# def S_preprocessing_data(text):

#     # Get English stop words
#     stop_words = set(stopwords.words('english'))

#     # Tokenize the text
#     word_tokens = word_tokenize(text)

#     # Filter out stop words
#     filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

#     # Join the filtered words into a string
#     filtered_sent = ' '.join(filtered_sentence)
#     #text = .strip(' ')

#     return filtered_sent

In [ ]:
import re
from nltk import word_tokenize
def preprocesing_text(text : str):
    try:
        text = text.lower()
        text = re.sub('[^a-zA-Z\s]', ' ', text)
        text = re.sub(r"(\s+|\t+|\r+|\n+)", " ",text)
        text = ' '.join([word for word in word_tokenize(text) if not word in stop_words])
        text = text.strip(' ')
        return text
    except Exception as e :
        print(e)
        return text

In [ ]:
train_data_csv.loc[:,'Abstract'] = train_data_csv['Abstract'].apply(preprocesing_text)

In [ ]:
train_data_csv.loc[:,'Title'] = train_data_csv['Title'].apply(preprocesing_text)

In [ ]:
print(train_data_csv['Abstract'])

In [ ]:
target_y = unique_tags
print(target_y)
target_y
print(target_y.shape)
target_y_list = target_y.tolist()
type(target_y_list)
print(target_y_list)
for category in target_y_list:
    train_data_csv[category] = train_data_csv['Categories'].apply(lambda x: 1 if category in x else 0)

In [ ]:
random_rows = train_data_csv.sample(30000)

In [ ]:
from sklearn.model_selection import train_test_split
train, validate = train_test_split(random_rows, test_size=0.1, shuffle=True)
train = train.reset_index(drop=True)
validate = validate.reset_index(drop=True)
train.shape, validate.shape

In [ ]:
class DataSetSentiment:
    def __init__(self,df,target_columns,tokenizer,max_len):
        self.titles = df['Title']
        self.abstracts = df['Abstract']
        self.targets = df[target_columns].values
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.titles)

    def __getitem__(self,item):
        title = str(self.titles[item])
        abstract = str(self.abstracts[item])
        target = self.targets[item]

        encoding_title = self.tokenizer.encode_plus(title,
                                          add_special_tokens=True,
                                          max_length=self.max_len,
                                          truncation=True,
                                          return_token_type_ids=False,
                                          padding='max_length',
                                          return_attention_mask=True,
                                          return_tensors='pt',
                                          )
        encoding_abstract = self.tokenizer.encode_plus(abstract,
                                          add_special_tokens=True,
                                          max_length=self.max_len,
                                                         truncation=True,
                                          return_token_type_ids=False,
                                          padding='max_length',
                                          return_attention_mask=True,
                                          return_tensors='pt',
                                          )

        return {'inp_id_title':torch.as_tensor(encoding_title['input_ids'], dtype=torch.long),
               'att_mk_title':torch.as_tensor(encoding_title['attention_mask'], dtype=torch.long),
               'inp_id_abstract':torch.as_tensor(encoding_abstract['input_ids'], dtype=torch.long),
               'att_mk_abstract':torch.as_tensor(encoding_abstract['attention_mask'], dtype=torch.long),
                'targets':target}

In [ ]:
from torch.utils.data import DataLoader

def create_dataloader(df,target_columns,tokenizer,max_len,params):
    ds = DataSetSentiment(df,target_columns,tokenizer,max_len)
    #print(ds)
    return DataLoader(ds,**params)

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
PARAMS = {'batch_size': 4,
          'shuffle': True,
          'num_workers': 0
          }

MAX_LEN = 512
EPOCHS = 5
LEARNING_RATE = 1e-05
#LEARNING_RATE = 0.0002

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
train_dataloader = create_dataloader(train,target_y_list,tokenizer,MAX_LEN,PARAMS)
validate_dataloader = create_dataloader(validate,target_y_list,tokenizer,MAX_LEN,PARAMS)

In [ ]:
data = next(iter(validate_dataloader))
data.keys()
print("Shape of inp_id_title:", data['inp_id_title'].shape)
print("Shape of inp_id_title:", data['att_mk_title'].shape)

In [ ]:
class DistilBERTModel(torch.nn.Module):
    def __init__(self,n_classes):
        super(DistilBERTModel,self).__init__()
        self.l1 = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.linear = torch.nn.Linear(768,384)
        #self.linear = torch.nn.Linear(192,96)
        self.dropout = torch.nn.Dropout(0.2)
        self.final = torch.nn.Linear(768,n_classes)

    def forward(self,inp_id_title,att_mk_title,inp_id_abstract,att_mk_abstract):
        title_l1 = self.l1(inp_id_title,att_mk_title)
        abstract_l1 = self.l1(inp_id_abstract,att_mk_abstract)

        pooler_title = title_l1[0][:, 0]
        pooler_abstract = abstract_l1[0][:, 0]

        pooler_title = self.linear(pooler_title)
        pooler_title = torch.nn.ReLU()(pooler_title)

        pooler_abstract = self.linear(pooler_abstract)
        pooler_abstract = torch.nn.ReLU()(pooler_abstract)

        combined_features = torch.concat([pooler_title, pooler_abstract], dim=1)

        pooler = self.dropout(combined_features)
        output = self.final(pooler)
        # output = torch.nn.Sigmoid()(output)
        return output

In [ ]:
model = DistilBERTModel(len(target_y_list))
model.to(device)

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = LEARNING_RATE)

In [ ]:

from sklearn.metrics import accuracy_score
def model_train(dataloader,device,params):
    losses = []
    acc = 0
    counter = 0
    model.train()

    for _,data in enumerate(dataloader,0):

        optimizer.zero_grad()

        title_ids = data['inp_id_title'].reshape(params['batch_size'],MAX_LEN).to(device,dtype=torch.long)
        title_am = data['att_mk_title'].to(device,dtype=torch.long)

        abstract_ids = data['inp_id_abstract'].reshape(params['batch_size'],MAX_LEN).to(device,dtype=torch.long)
        abstract_am = data['att_mk_abstract'].to(device,dtype=torch.long)

        targets = data['targets'].to(device,dtype=torch.float)

        outputs = model(title_ids,title_am,abstract_ids,abstract_am)
        loss = loss_function(outputs,targets)
        predictions = torch.as_tensor(outputs.data>0.5,dtype=torch.float)

        targets = targets.cpu().detach().numpy()
        predictions = predictions.cpu().detach().numpy()
        losses.append(loss.item())
        acc += accuracy_score(targets,predictions.round(), normalize=False)

        loss.backward()
        optimizer.step()

        counter+=1

    return acc/counter, np.mean(losses)

In [ ]:
for epoch in range(EPOCHS):
    accuracy, loss = model_train(train_dataloader,device,PARAMS)
    print(f"Epoch - {epoch}, Accuracy - {accuracy}, Loss - {loss}")

In [ ]:
def model_eval(dataloader,device,params):
    losses = []
    acc = 0
    counter = 0
    model.eval()

    with torch.no_grad():
        for _,data in enumerate(dataloader,0):

            title_ids = data['inp_id_title'].reshape(params['batch_size'],MAX_LEN).to(device,dtype=torch.long)
            title_am = data['att_mk_title'].to(device,dtype=torch.long)

            abstract_ids = data['inp_id_abstract'].reshape(params['batch_size'],MAX_LEN).to(device,dtype=torch.long)
            abstract_am = data['att_mk_abstract'].to(device,dtype=torch.long)

            targets = data['targets'].to(device,dtype=torch.float)

            outputs = model(title_ids,title_am,abstract_ids,abstract_am)
            loss = loss_function(outputs,targets)
            predictions = torch.as_tensor(outputs.data>0.5,dtype=torch.float)

            targets = targets.cpu().detach().numpy()
            predictions = predictions.cpu().detach().numpy()
            losses.append(loss.item())
            acc += accuracy_score(targets,predictions.round(), normalize=False)

            counter+=1
        return acc/counter, np.mean(losses)

In [ ]:
acc, loss = model_eval(validate_dataloader, device, PARAMS)
acc, loss

In [ ]:
model_save_name='Kriti2024_3rd_model.pt'
path=F"/kaggle/working/{model_save_name}"
torch.save(model.state_dict(),path)